In [1]:
!pip install requests

In [2]:
import requests
import re

In [3]:
def links_collect_esenin():
  result = requests.get('https://ilibrary.ru/author/esenin/l.all/index.html')
  prob = result.text
  links = re.search('<span class="azh">Полный&nbsp;список</span></div><span id="text"><div style="margin: 0em 0em 2em">(.*?)<div style', prob, flags=re.DOTALL)
  l = links.group(0)
  all_li = re.compile('<a href="(.*?)">', flags=re.DOTALL)
  links = all_li.findall(l)
  li = []
  for i in links:
    li.append('https://ilibrary.ru'+i[:-10]+'p.1/index.html')
  return(li)

In [4]:
def links_collect_krug():
  result = requests.get('https://www.mkrug.ru/txt/')
  #result.encoding = 'utf-8'
  prob = result.text
  links = re.findall('<p><a href=(.*?)>(.*?)</a>', prob, flags=re.DOTALL)
  l = [[a[0], a[1]] for a in links]
  li = []
  for i in l:
    li.append(['https://www.mkrug.ru/txt/'+i[0], i[1]])
  return(li)

In [5]:
l_e = links_collect_esenin()
l_k = links_collect_krug()

In [7]:
def htmls_collect(links):
  htmls = []
  for i in links:
    result = requests.get(i)
    prob = result.text
    #print(i)
    htmls.append(prob)
  return(htmls)

In [10]:
e_htmls = htmls_collect(l_e)
k_htmls = htmls_collect([l[0] for l in l_k])

In [17]:
def e_texts_get(htmls):
  texts = []
  for i in htmls:
    if 'стр. ' not in i:
      t = re.search('<span class="vl">(.*?)<div id="tbd">', i, flags=re.DOTALL).group(0)
      word = re.compile('>(.*?)<')
      words = word.findall(t)
      words2 = [word for word in words if word != '']
      text = ' '.join(words2)
      name = re.search('<TITLE>(.*?)</TITLE>', i, flags=re.DOTALL).group(0)[7:-27]
      name = re.sub('&.*?;', '', name)
      text = re.sub('&.*?;', '', text)
      text = re.sub('.....&mdash;', '', text)
      text = re.sub('poems.push.*?;', '', text)
      nameyear = name+' '+text[-5:-1]
      text = text[:-6]
      flag = False
      for j in texts:
        if text[20:40] in j[1]:
          flag = True
      if flag == False:
        #print(nameyear)
        texts.append([nameyear, text])
  return(texts)    

In [22]:
def k_texts_get(htmls):
  texts = []
  for i in htmls:
      t = re.search('<pre>(.*?)</pre>', i, flags=re.DOTALL).group(0)
      x = t[5:]
      y = x[:-6]
      text = re.sub('Припев.*?\n', '', y)
      text1 = re.sub('\r\n', ' ', text)
      texts.append(text1)
  krug = []
  for i in range(len(texts)):
    krug.append(['Михаил Круг: '+l_k[i][1], texts[i]])
  return(krug)    

In [23]:
e_texts = e_texts_get(e_htmls)
k_texts = k_texts_get(k_htmls)

In [25]:
e_texts[0]

['С. А. Есенин. Алый мрак в небесной черни.... 1915',
 'Алый мрак в небесной черни Начертил пожаром грань. Я пришел к твоей вечерне, Полевая глухомань. Нелегка моя кошница, Но глаза синее дня. Знаю, мать-земля черница, Все мы тесная родня. Разошлись мы в даль и шири Под лазоревым крылом. Но сзовет нас из псалтыри Заревой заре псалом. И придем мы по равнинам К правде сошьего креста Светом книги голубиной Напоить свои уста. ']

In [ ]:
krug = []
for i in range(len(ktexts)):
  krug.append(['Михаил Круг: '+krugli[i][1], ktexts[i]])

print(krug[0])

['Михаил Круг: 1, 2, 3 по почкам', ' Ну кто бы знал, что это мент, ведь он был в штатском,  А Осип зарядил не разобрав в пылу.  И не успел он по карманам разобраться  У рaзвалившегося дяди на полу.  Как вдруг откуда-то братья его по классу,  И ну выкручивать суставы за спиной.  Он пожалел, что зарядил менту с атаса,  Уже когда попал в клоповник мусоркой.      Раз, два, три по почкам,   Раз, два, три попечени,  Потерпи браточек,   А мы тебя подлечим.  Раз, два, три по почкам,   Раз, два, три попечени,  Потерпи браточек,   Уж они тебя подлечат.   Там били так, что пару раз он отключался,  И от ударов снова приходил в себя.  А дай в обратную, так там бы о остался,  Его скормили бы как булку голубям.  И потоптались там по ребрышкам жигана,   И вбили ненависть к кокарде мусорской.  И если Ося бъет кому в табло по пъяни,   То без свидетелей и тьфу,тьфу,тьфу живой!   ']


In [20]:
k_texts[0]

' Ну кто бы знал, что это мент, ведь он был в штатском,  А Осип зарядил не разобрав в пылу.  И не успел он по карманам разобраться  У рaзвалившегося дяди на полу.  Как вдруг откуда-то братья его по классу,  И ну выкручивать суставы за спиной.  Он пожалел, что зарядил менту с атаса,  Уже когда попал в клоповник мусоркой.      Раз, два, три по почкам,   Раз, два, три попечени,  Потерпи браточек,   А мы тебя подлечим.  Раз, два, три по почкам,   Раз, два, три попечени,  Потерпи браточек,   Уж они тебя подлечат.   Там били так, что пару раз он отключался,  И от ударов снова приходил в себя.  А дай в обратную, так там бы о остался,  Его скормили бы как булку голубям.  И потоптались там по ребрышкам жигана,   И вбили ненависть к кокарде мусорской.  И если Ося бъет кому в табло по пъяни,   То без свидетелей и тьфу,тьфу,тьфу живой!   '

In [26]:
with open('esenin.txt', 'w', encoding='utf-8') as f:
  for i in e_texts:
    f.write(i[0]+'\n'+i[1]+'\n\n')

In [28]:
with open('krug.txt', 'w', encoding='utf-8') as f:
  for i in k_texts:
    f.write(i[0]+'\n'+i[1]+'\n\n')

In [29]:
from nltk import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
def sentsmaker(info):
  sents = []
  for i in info:
    sents.append([i[0], sent_tokenize(i[1])])
  return(sents)

In [31]:
krugsents = sentsmaker(k_texts)
eseninsents = sentsmaker(e_texts)

In [32]:
!pip install git+https://github.com/nlpub/pymystem3

  Cloning https://github.com/nlpub/pymystem3 to /tmp/pip-req-build-kg879nj8
  Running command git clone -q https://github.com/nlpub/pymystem3 /tmp/pip-req-build-kg879nj8
  Created wheel for pymystem3: filename=pymystem3-0.2.0-cp36-none-any.whl size=9922 sha256=581c3636c246e781ea366cf6d199a73e036149cc23edd12900f8141d9e1a3563
  Stored in directory: /tmp/pip-ephem-wheel-cache-9exz4fj_/wheels/7d/75/c2/216a594291dee680749ce12c60d16125cfe1f363059e7163dc
Successfully built pymystem3


In [33]:
from pymystem3 import Mystem
m = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [34]:
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2020-10-22 10:43:56--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.245, 5.45.205.241, 5.45.205.244, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.245|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskm906.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-10-22 10:43:57--  http://cache-mskm906.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskm906.cdn.yandex.net (cache-mskm906.cdn.yandex.net)... 5.45.220.16, 2a02:6b8:0:2002::17
Connecting to cache-mskm906.cdn.yandex.net (cache-mskm906.cdn.yandex.net)|5.45.220.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz’

mystem-3.0-linux3.1 100%[===================>]  1

In [63]:
def makedata(sent):
  sentwp = []
  ana = m.analyze(sent)
  for word in ana:
    #print(word)
    if 'analysis' in word:
      if word['analysis'] != []:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        lex = word['analysis'][0]['lex']  
        form = word['text'].lower()
      else:
        pos = 'null'
        lex = 'null'
        form = 'null'
      sentwp.append([lex, pos, form])
  #print(sentwp)
  lexems = []
  poses = []
  forms = []
  for i in sentwp:
    lexems.append(i[0])
    poses.append(i[1])
    forms.append(i[2])
  full = [sent, lexems, poses, forms]
  return full

In [64]:
def allinone(sents):
  allinfos = []
  for i in sents:
    sentinfos = []
    for j in i[1]:
      sentinfo = makedata(j)
      sentinfo.append(i[0])
      sentinfos.append(sentinfo)
    allinfos.extend(sentinfos)
  return(allinfos)

In [65]:
allkrug = allinone(krugsents)
allesenin = allinone(eseninsents)

In [66]:
corpus = allkrug + allesenin

In [67]:
corpus[0]

[' Ну кто бы знал, что это мент, ведь он был в штатском,  А Осип зарядил не разобрав в пылу.',
 ['ну',
  'кто',
  'бы',
  'знать',
  'что',
  'это',
  'мент',
  'ведь',
  'он',
  'быть',
  'в',
  'штатское',
  'а',
  'осип',
  'заряжать',
  'не',
  'разбирать',
  'в',
  'пыл'],
 ['PART',
  'SPRO',
  'PART',
  'V',
  'CONJ',
  'SPRO',
  'S',
  'CONJ',
  'SPRO',
  'V',
  'PR',
  'S',
  'CONJ',
  'S',
  'V',
  'PART',
  'V',
  'PR',
  'S'],
 ['ну',
  'кто',
  'бы',
  'знал',
  'что',
  'это',
  'мент',
  'ведь',
  'он',
  'был',
  'в',
  'штатском',
  'а',
  'осип',
  'зарядил',
  'не',
  'разобрав',
  'в',
  'пылу'],
 'Михаил Круг: 1, 2, 3 по почкам']

In [68]:
corpus = [i for i in corpus if len(i[1])<40 and i[4] != 'Михаил Круг: Селигер' and i[4] != 'С. А. Есенин. Ключи Марии. ина.']

In [69]:
with open('corpus.txt', 'w', encoding='utf-8') as f:
  for sent in corpus:
    f.write(sent[0]+'\n')
    for word in sent[1]:
      f.write(word+' ')
    f.write('\n')
    for pos in sent[2]:
      f.write(pos+' ')
    f.write('\n')
    for form in sent[3]:
      f.write(form+' ')
    f.write('\n'+sent[4]+'\n\n')

In [72]:
with open('corpus.txt', encoding='utf-8') as f:
  line = f.read()

import re
line = re.sub(' \n', '\n', line)

corp = []
sents = line.split('\n\n')
for sent in sents:
  chunks = sent.split('\n')
  if len(chunks) == 5:
    l = []
    l.append(chunks[0])
    lexemes = chunks[1].split(' ')
    l.append(lexemes)
    poses = chunks[2].split(' ')
    l.append(poses)
    forms = chunks[3].split(' ')
    l.append(forms)
    l.append(chunks[4])
    corp.append(l)